In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra?
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sentiment140


In [3]:
import pandas as pd

df=pd.read_csv("/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv",encoding="latin-1",header=None)



In [5]:
df.columns=["target","ids","date","flag","user","text"]

df.head()

df.drop(["ids","date","flag","user"],axis=1,inplace=True)

df['target']=df['target'].replace(4,1)

df.target.value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [6]:
df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [7]:
import pandas as pd
import nltk
import re
import string

# ... (rest of your code) ...

def clean_text(tweet):
    # Check if the input is a string, convert if not
    if not isinstance(tweet, str):
        tweet = str(tweet)
    
    tweet= re.sub('(#|@)\w*','',tweet)
    tweet= re.sub('https?:\/\/\S+','',tweet)
    tweet= re.sub('(\?|\!)+','',tweet)
    tweet= re.sub('^\s+','',tweet)
    tweet= re.sub('\s+$','',tweet)
    tweet= re.sub('(\.|\,)+','',tweet)
    tweet= re.sub('[:()\\\]','',tweet)
    tweet= re.sub('\s\d+\s','',tweet)
    tweet = tweet.lower()  
    tweet = re.sub(r"\d+", "", tweet) 
    tweet = tweet.translate(str.maketrans("", "", string.punctuation)) 
    tweet = tweet.strip()
    return tweet



In [8]:
df["text"] = df['text'].apply(clean_text)


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import gc
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [10]:
# ✅ تحميل Tokenizer الخاص بـ RoBERTa
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# ✅ دالة تحويل النصوص إلى توكينات
def tokenize_texts(texts, max_length=128):
    encoding = tokenizer(
        texts.tolist(),
        
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    return encoding["input_ids"], encoding["attention_mask"]



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
# ✅ تحويل النصوص إلى توكينات

input_ids, attention_masks = tokenize_texts(df["text"])

In [12]:

labels = torch.tensor(df["target"].values)


# ✅ تنظيف الذاكرة
gc.collect()

0

In [13]:
from torch.utils.data import Dataset, DataLoader

# ✅ تقسيم البيانات إلى تدريب واختبار
X_train, X_test, y_train, y_test, mask_train, mask_test = train_test_split(
    input_ids, labels, attention_masks, test_size=0.2, random_state=42
)

# ✅ إنشاء Dataset مخصص
class SentimentDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_masks[idx],
            "labels": self.labels[idx]
        }

# ✅ إنشاء DataLoader
batch_size = 16  # اختبر قيمًا أكبر لو كان الأداء جيدًا
train_dataset = SentimentDataset(X_train, mask_train, y_train)
test_dataset = SentimentDataset(X_test, mask_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
from tqdm import tqdm


In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")


🚀 Using device: cuda


In [36]:
# ✅ تحميل نموذج RoBERTa جاهز للتصنيف
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

# ✅ إعداد الـ Optimizer & Loss
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
def train_fn(train_loader, model, optimizer, num_epochs=3, patience=2,
             load_checkpoint_path="checkpoint.pt", save_checkpoint_path="checkpoint_new.pt"):
    model.to(device)
    
    best_loss = float('inf')
    counter = 0
    start_epoch = 0

    # ✅ لو في Checkpoint محفوظ، نحمله من ملف مختلف
    if os.path.exists(load_checkpoint_path):
        checkpoint = torch.load(load_checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_loss = checkpoint['best_loss']
        print(f"✅ تم استعادة التدريب من Epoch {start_epoch}")

    for epoch in range(start_epoch, num_epochs):
        model.train()
        total_loss = 0
        start_time = time.time()

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"📌 Epoch {epoch+1}: Average Loss = {avg_loss:.4f}, Time = {(time.time() - start_time) / 60:.2f} دقيقة")

        # ✅ حفظ Checkpoint في فايل جديد
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_loss': best_loss,
        }, save_checkpoint_path)
        print(f"💾 تم حفظ Checkpoint عند Epoch {epoch+1} في {save_checkpoint_path}")

        # ✅ Early Stopping
        if avg_loss < best_loss:
            best_loss = avg_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("⏳ Early Stopping: No improvement, stopping training.")
                break


In [ ]:
train_fn(train_loader, model, optimizer, num_epochs=2, patience=2, checkpoint_path="checkpoint.pt")
        

Epoch 1/2: 100%|██████████| 80000/80000 [8:32:23<00:00,  2.60it/s]  


📌 Epoch 1: Average Loss = 0.3376, Time = 512.39 دقيقة
💾 تم حفظ Checkpoint عند Epoch 1


Epoch 2/2:  10%|█         | 8398/80000 [53:48<7:39:45,  2.60it/s]

In [ ]:
import shutil

# ✅ حفظ النموذج
model_save_path = "roberta_sentiment_model.pt"
torch.save(model.state_dict(), model_save_path)

# ✅ إذا كنت في Kaggle أو Colab، قم بنقل الملف إلى مجلد العمل
shutil.move(model_save_path, "/kaggle/working/roberta_sentiment_model.pt")  # لكاجل


print(f"✅ تم حفظ النموذج في {model_save_path}")


In [ ]:
import os

# إنشاء مجلد لحفظ النموذج
model_dir = "/kaggle/working/roberta_trained"
os.makedirs(model_dir, exist_ok=True)

# ✅ حفظ النموذج والتوكينايزر
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

print(f"✅ تم حفظ النموذج والتوكينايزر في: {model_dir}")


In [ ]:
import torch
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ✅ تحويل الموديل إلى وضع التقييم
model.eval()

# ✅ تخزين التوقعات والملصقات الحقيقية
predictions = []
true_labels = []

# ✅ عدم احتساب الجريدينت أثناء التقييم
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)


In [ ]:
accuracy = accuracy_score(true_labels, predictions)
print(f"🎯 دقة النموذج: {accuracy:.4f}")


In [ ]:
print("📊 تقرير الأداء:\n")
print(classification_report(true_labels, predictions, target_names=["neg", "natural", "pos"]))  # استبدل الفئات بأسماء التصنيفات الفعلية لديك


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

cm = confusion_matrix(true_labels, predictions)
labels = ["neg", "natural", "pos"]  # استبدلها بالفئات الصحيحة لديك

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.xlabel("pred")
plt.ylabel("true")
plt.title("📊 confusion metrix")
plt.show()


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

# لو كنت لسه ما حمّلتش الـ model
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=3)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# تحميل الـ checkpoint
checkpoint = torch.load("/kaggle/working/checkpoint.pt")  # أو المسار المناسب عندك

# استرجاع الحالة
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch'] + 1
best_loss = checkpoint['best_loss']

print(f"✅ تم استعادة النموذج من epoch رقم {start_epoch} - أفضل Loss: {best_loss}")
